# Week 11 Statistical Inference and Regression Assumptions Lecture Demo

This notebook contains various examples to demonstrate the mehods and approches of examining the theoretical concepts of my this week lecture in Python.

In [201]:
# Importing all the libraries we will use in this demo

import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # using package of testing VIF in statsmodels
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import numpy as np

## 1.0 Regression Assumptions

In this notebook, we will demonstrate the following regression assumptions discussed in the Week-11 lecture:

**Assumption 2 - For any values of the explanatory variables, the variance (or standard deviation) of the dependent variable is a constant, the same for all such values.**

**Assumption 3 - For any values of the explanatory variables, the dependent variable is normally distributed.**





<Center><Font Size="5">Assumption 2</Font></Center>




In [202]:
# Detecting nonconstant error variance through a visual inspection
# Here we will take the week-8 examples of Pharmex and Bendrix and an example violates the assumption 'marketing catalog'

df_Pharmex = pd.read_csv('../data/Lecture_1_Drugstore Sales.csv')
df_Bendrix = pd.read_csv('../data/Lecture_2_Overhead Costs.csv')
df_Catalog = pd.read_csv('../data/Lecture_5_Catalog Marketing.csv')


display(df_Pharmex.head())
display(df_Bendrix.head())
display(df_Catalog.head())

,Region,Promote,Sales
0,1,77,85
1,2,110,103
2,3,110,102
3,4,93,109
4,5,90,85


,Month,Machine Hours,Production Runs,Overhead
0,1,1539,31,99798
1,2,1284,29,87804
2,3,1490,27,93681
3,4,1355,22,82262
4,5,1500,35,106968


,Person,Age,Gender,Own Home,Married,Close,Salary,Children,History,Catalogs,Region,State,City,First Purchase,Amount Spent,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,1,1,0,0,0,1,"$16,400",1,1.0,12,South,Florida,Orlando,23/10/2011,$218,NaN,NaN,NaN,NaN
1,2,2,0,1,1,0,"$108,100",3,3.0,18,Midwest,Illinois,Chicago,25/05/2009,"$2,632",NaN,NaN,NaN,NaN
2,3,2,1,1,1,1,"$97,300",1,NaN,12,South,Florida,Orlando,18/08/2015,"$3,048",NaN,NaN,NaN,NaN
3,4,3,1,1,1,1,"$26,800",0,1.0,12,East,Ohio,Cleveland,26/12/2012,$435,NaN,NaN,NaN,NaN
4,5,1,1,0,0,1,"$11,200",0,NaN,6,Midwest,Illinois,Chicago,04/08/2015,$106,NaN,NaN,NaN,NaN


In [ ]:
# Rename the variable name with space
df_Catalog = df_Catalog.rename(columns={'Amount Spent':'AmountSpent'})

# Preparing the data for further analysis and illustration
def format_Salary(Salary):
    return(int(Salary.replace('$','').replace(',','')))

def format_Spent(AmountSpent):
    return(int(AmountSpent.replace('$','').replace(',','')))

df_Catalog['SalaryInt'] = df_Catalog['Salary'].apply(format_Salary)
df_Catalog['SpentInt'] = df_Catalog['AmountSpent'].apply(format_Spent)

In [ ]:
# Illustrating the scatterplots for both examples
# For the Pharmex, we examine the relationship between Sales and Value of Promote
# For the Bendrix, we examine the relationship Overhead and number of machine hours

fig = plt.figure()
fig.set_size_inches(30, 20)
fig.set_dpi(100)

axes1 = fig.add_subplot(2,2,1)
axes2 = fig.add_subplot(2,2,2)
axes3 = fig.add_subplot(2,2,3)
axes4 = fig.add_subplot(2,2,4)

axes1.plot(df_Bendrix['Machine Hours'], df_Bendrix['Overhead'], 'o')
axes2.plot(df_Bendrix['Production Runs'], df_Bendrix['Overhead'], 'o')
axes3.plot(df_Pharmex['Promote'], df_Pharmex['Sales'], 'o')
axes4.plot(df_Catalog['SalaryInt'], df_Catalog['SpentInt'], 'o')

axes1.set_title('Scatterplot of Machine Hours Versus Overhead', pad=30, fontsize=17)
axes1.set_xlabel('Machine Hours', fontsize=15)
axes1.set_ylabel('Overhead', fontsize=15)

axes2.set_title('Scatterplot of Production Runs Versus Overhead', pad=30, fontsize=17)
axes2.set_xlabel('Production Runs', fontsize=15)
axes2.set_ylabel('Overhead', fontsize=15)

axes3.set_title('Scatterplot of Promote Versus Salex', pad=30, fontsize=17)
axes3.set_xlabel('Promote', fontsize=15)
axes3.set_ylabel('Sales', fontsize=15)

axes4.set_title('Scatterplot of Salary Versus Spent', pad=30, fontsize=17)
axes4.set_xlabel('Salary', fontsize=15)
axes4.set_ylabel('Spent', fontsize=15)

plt.subplots_adjust(wspace=0.2, 
                    hspace=0.2)
plt.show()

### 1.1.1. Explaination of the Assumption 2 Demo

In the Pharmex example, constant error variance implies that the variation in Sales values is the same regardless of the value of Promote. As another example, the Bendrix manufacturing example. There were relationships overhead costs (Overhead) to the number of machine hours (Machine Hours) and the number of production runs (Production Runs). Constant error variance implies that overhead costs vary just as much for small values of Machine Hours and Production Runs as for large values—or any values in between.

In addition, **Scatterplot of Salary Versus Spent** shows the amount spent versus salary for a sample ofthe company’s customers. Clearly, the variation in the amount spent increases as salaryincreases, which makes intuitive sense. Customers with small salaries have little dispos-able income, so they tend to spend small amounts at online stores. Customers with large salaries have more disposable income. Some of them spend a lot of it at online stores and some spend only a little of it—hence, a larger variation. 

Scatterplots with this “fan” shapeare not uncommon in real studies, and they exhibit a clear violation of assumption 2. We say that the data in this graph exhibit heteroscedasticity, or more simply, nonconstanterror variance.

In [ ]:
# Lograithmic transformation of the dependent variable can help to solve the violation of Assumption 2
# Taking the example of Salary Versus Spent
# We can use logarithmic transormation the amount Spent

df_Catalog['Log_Spent'] = np.log(df_Catalog['SpentInt'])

# Plotting the new scatterplot based on the log transformed variable

fig = plt.figure()
fig.set_size_inches(12, 6) # Adjusting height and width of the figure
fig.set_dpi(100)

plt.scatter(df_Catalog['SalaryInt'], df_Catalog['Log_Spent'], s=15)

plt.title('Adjustment', fontsize=24)
plt.xlabel('Salary', fontsize=14)
plt.ylabel('Log_Spent', fontsize=14)

plt.show()


### 1.1.2. Explaination of the Assumption 2 Adjustment Demo


When you see a fan shape, where the variability increases from left to right in a scatterplot, you can try a logarithmic transformation of the dependent variable. The reason this often works is that the logarithmic transformation squeezes the large values closer together and pulls the small values farther apart. The scatterplot of the log of Amount Spent versus Salary is in above figure. Clearly, the fan shapeevident in Figure of `Scater plot of Salary versus Spent` is gone.




<Center><Font Size="5">Assumption 3</Font></Center>




In [ ]:
# Taking the Pharmex as an example

model = smf.ols(formula='Sales ~ Promote', data=df_Pharmex).fit()

sns.histplot(model.resid) # We can simply pass the histplot() function the array of residuals

### 1.2.1. Explaination of the Assumption 3

Assumption 3 is equivalent to stating that the errors are normally distributed. You can check this by forming a histogram of the residuals. If assumption 3 holds, the histogram should be approximately symmetric and bell-shaped.

The above histagram indicates the assumption is not violated. 

## 2.0 Multicollinearity

We used a different dataframe of employee's salary to demonstrate this issue.

In [ ]:
# importing the example data
df = pd.read_csv('../data/Lecture_6_Salary_2.csv')
df_Salary=df.dropna(how='all') # Removing the observations with missing values
df_Salary = pd.get_dummies(df_Salary, columns=['Gender'])
df_Salary

In [ ]:
# We try another way to build the regression model that without using the formula api
# Case sensitive for the names of dataframe

y = df_Salary.Salary
X = df_Salary[["Gender_Female","Age","Experience","Seniority"]].assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

In [ ]:
# using the vif function to obtain the vif value of the independent variables

vif = [variance_inflation_factor(exog=X.values, exog_idx=i) for i in range(X.shape[1])]

# creating a new dataframe to report the results
vif_table = pd.DataFrame({'coef_name': X.columns, 'vif': np.around(vif,3)})
print(vif_table)

# checking the correlation table to reconfirm the results
X.corr(method = 'pearson')

### 2.1 Multicollinearity Rules of Thumb

A VIF of 1 means that there is no correlation among the k-th predictor and the remaining predictorvariables, and hence the variance of k-th regression coefficient is not inflated at all. The general rule of thumb is that VIFs exceeding 4 warrant further investigation, while VIFs exceeding 10 are signs of serious multicollinearity requiring correction.

In [ ]:
# To improve the model, we need remove the variables with multicollinearity issues

estimation = smf.ols(formula='Salary~Gender_Female+Seniority', data=df_Salary).fit()
print(estimation.summary())

## 3.0 Outliers

In this example, we will locate possible outliers in the bank salary data, and to see to what extent they affect the regression model.

In [ ]:
df_Bank = pd.read_csv('../data/Lecture_3_Bank Salaries.csv')

def format_Salary(Salary):
    return(int(Salary.replace('$','').replace(',','')))

df_Bank['SalaryInt'] = df_Bank['Salary'].apply(format_Salary)

df_Bank.head()

In [ ]:
# changing color for the boxplot color
# please note that seaborn uses a method that is a function of the inter-quartile range
# the below numerical way (3.1) about the 3-sigma is a different approch to detect outliers

boxplot = sns.boxplot(x='SalaryInt', data=df_Bank, color='green')


In [ ]:
df_Bank_Boxplot_Method = df_Bank.drop(df_Bank[df_Bank.SalaryInt > 60000].index)
boxplot2 = sns.boxplot(x='SalaryInt', data=df_Bank_Boxplot_Method, color='yellow')

### 3.1. Numerical way to identify the outliers

The data points which fall below mean-3*(sigma) or above mean+3*(sigma) are outliers.

where mean and sigma are the average value and standard deviation of a particular column.

<center><img src="../Image/Deviations.png" width=600 height=400 /></center>

In [203]:
# definint the boundaries to identify the outliers through the 3-sigma approach

a = df_Bank['SalaryInt'].mean()
b = df_Bank['SalaryInt'].std()
(a+3*b, a-3*b)

(73690.3848427993, 6153.461311046849)

In [ ]:
# identify the rows of df_Bank that are out of the boundaries

df_Bank[(df_Bank['SalaryInt']>73690.3848427993)|(df_Bank['SalaryInt']<6153.461311046849)]['SalaryInt']

In [ ]:
# dropping the identified observations with outliers

df_Bank_no_Outliers = df_Bank.drop([202, 203, 204, 205, 206])

boxplot = sns.boxplot(x='SalaryInt', data=df_Bank_no_Outliers, color='pink')